# Construindo um Sistema de Recomendação do zero
Slides: https://bit.ly/2lIFyIn

Teoria e prática na construção de sistemas de recomendação escaláveis utilizando tecnologias de Big Data e Machine Learning. O Workshop visa o passo a passo desde a teoría por trás de RecSys, algoritmos utilizados, ferramentas e como recomendar conteúdo relevante.


![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-01.jpg)


![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-04.jpg?1=1)


![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-02.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-03.jpg?a=1)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-05.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-06.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-07.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-08.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-09.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-10.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-11.jpg)

## Collaborative Filtering

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-12.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-13.jpg)

In [0]:
# Dados históricos de interação do USUARIO com o ITEM contendo o Rating (Nota como preferência explicita)
#
#
import pandas as pd
pd.options.display.precision = 3


hist = pd.DataFrame([
         ("Judite", "Melancia", 7),
         ("Judite", "Pizza", 10),
         ("Moises", "Pizza", 8),
         ("Moises", "Hamburguer", 9),
         ("Moises", "Melancia", 6),
         ("Rafael", "Pizza", 1),
         ("Rafael", "Hamburguer", 3),
         ("Rafael", "Melancia", 10),
      ], columns=["Usuario", "Item", "Nota"])

hist

In [0]:
# Matriz de Interação USUARIO X ITEM
#
# A Matriz de interação contem todas as interações dos usuários (linhas) com os itens (colunas)
# em que o valor é o rating dado pelo USUARIO ao ITEM 
#
cf = hist.pivot(index='Usuario', columns='Item', values='Nota')
cf = cf[['Pizza', 'Hamburguer', 'Melancia']]
cf

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-14.jpg)

In [0]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
from sklearn.metrics.pairwise import pairwise_distances

# Vetor 1
v1 = [[10, 0, 7]] 

# Vetor 2
v2 = [[8,  9, 6]]

print("\nEuclidean:\n",    pairwise_distances(v1, v2, metric='euclidean'))
print("\nCosine:\n",       pairwise_distances(v1, v2, metric='cosine'))
print("\nCorrelation:\n",  pairwise_distances(v1, v2, metric='correlation'))

### Distância Euclidiana

In [0]:
# Distância Euclidiana da Matriz de Interação
#
#

d_cf = pairwise_distances(cf.fillna(cf.mean()), metric='euclidean')
d_cf = pd.DataFrame(d_cf, columns = cf.index, index=cf.index)
d_cf

### Distância do Coseno Inversa (Similaridade)

In [0]:
# Similaridade do Coseno da Matriz de Interação
#
#

d_cf = 1 - pairwise_distances(cf.fillna(cf.mean()), metric='cosine')
d_cf = pd.DataFrame(d_cf, columns = cf.index, index=cf.index)
d_cf

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-15.jpg)

## Content Based

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-16.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-17.jpg)

In [0]:
# Dados de características (features) dos ITEMS
#
#

itens = pd.DataFrame([
         ("Pizza",      True, True, False, False),
         ("Hamburguer", True, True, False, False),
         ("Melancia",   False, False, True, True),
      ], columns=["Item", "Junk Food", "Tem Queijo", "Doce", "Natural"]).set_index('Item')

itens

In [0]:
itens.loc[['Pizza']]

In [0]:
# Similaridade do Coseno da Matriz de Itens
#
#
import numpy as np

sim = 1 - pairwise_distances(itens.loc[['Pizza']], itens.loc[['Hamburguer']], metric='cosine')
print("Similaridade:", sim.reshape(-1)[0])

#pairwise_distances(itens.loc[['Pizza']], itens.loc[['Melancia']], metric='cosine')

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-18.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-19.jpg)

In [0]:
# O Cold Started pode ser realizado a partir do historico geral de interações
# Dessa forma é possível gerar listas usando algumas heurísticas
#

hist

In [0]:
# A depender da estatística a lista de recomendação pdoe mudar..
#  .count()
#  .mean()
#  .median()
#
hist.groupby(['Item'])\
    .count()\
    .sort_values('Nota', ascending=False)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-20.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-21.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-22.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-23.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-24.jpg)

![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-25.jpg)

# Recomendação de Filmes com Content-Based e Collaborative Filtering

**“*Que filme devo assistir esta noite?*”**

Você já teve que responder a essa pergunta pelo menos uma vez quando chegou em casa do trabalho? 
Do Netflix ao Youtube, a necessidade de construir sistemas robustos de recomendação de filmes é extremamente importante, dada a enorme demanda por conteúdo personalizado e a grande quantidade de conteúdo disponível.

Um exemplo de funcionamento de RecSys:
* O usuário A assiste **Game of Thrones** e **Breaking Bad**.
* O usuário B pesquisa em **Game of Thrones** e, em seguida, o sistema sugere **Breaking Bad** a partir de dados coletados sobre o usuário A.

Os sistemas de recomendação são utilizados não apenas para filmes, mas em vários outros produtos e serviços, como Amazon (Livros, Itens), Pandora / Spotify (Música), Google (Notícias, Pesquisa), YouTube (Vídeos) etc.

![netflix](https://github.com/khanhnamle1994/movielens/raw/cb1fe40c99cdd61c3c714e501e11f699c87b0eed/images/netflix.png)


Os dois tipos mais utilizados de sistemas de recomendação são baseados em **Conteúdo** e **Filtragem colaborativa**. A filtragem colaborativa produz recomendações com base no conhecimento da atitude dos usuários em relação aos itens, ou seja, usa a "sabedoria da multidão" para recomendar itens. Por outro lado, os sistemas de recomendação baseados em conteúdo concentram-se nos atributos dos itens e fornecem recomendações com base na semelhança entre eles.

Neste notebook, tentaremos implementar esses dois sistemas para recomendar filmes e avaliá-los para ver qual deles apresenta melhor desempenho.

**Depois do workshop, você saberá:**

* Sobre o Dataset MovieLens, bastante utilizado em sistema de recomendação.
* Como carregar e processar os dados.
* Como fazer análise exploratória de dados.
* Os 2 tipos diferentes de mecanismos de recomendação.
* Como desenvolver um modelo de recomendação baseado em conteúdo com base em gêneros de filmes.
* Como desenvolver um modelo de filtragem colaborativa com base nas classificações do usuário.
* Abordagem alternativa para melhorar os modelos existentes.

## O Dataset MovieLens

Um dos DataSets comuns disponíveis na Internet para a construção de um sistema de recomendação é o [MovieLens DataSet] (https://grouplens.org/datasets/movielens/). Esta versão do conjunto de dados com que estamos trabalhando ([1M] (https://grouplens.org/datasets/movielens/1m/)) contém 1.000.209 classificações anônimas de aproximadamente 3.900 filmes feitos por 6.040 usuários.

![movielens](https://github.com/khanhnamle1994/movielens/raw/cb1fe40c99cdd61c3c714e501e11f699c87b0eed/images/movielens.jpg)

## Preparação do Dataset

Vamos carregar os dados no Python. A informações utilizadas são as interações de avaliação do usuário para o filme, os dados do usuário e os dados dos filmes.

* Rating
* Usuarios
* Filmes 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import seaborn library
import seaborn as sns
%matplotlib inline
sns.set(style="white")
sns.set(font_scale=1.5)

### Ratings Dataset

In [0]:
# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/dataset/ratings.csv', 
                      sep='\t', encoding='latin-1', 
                      usecols=['user_id', 'movie_id', 'rating'])

In [0]:
# Check the top 5 rows
ratings.head()

In [0]:
# Check the file info
print(ratings.shape)

Isso confirma que existem classificações de 1 milhão para diferentes combinações de usuário e filme.

### Users Dataset

In [0]:
# Reading users file
users = pd.read_csv('https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/dataset/users.csv', 
                    sep='\t', encoding='latin-1', 
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

In [0]:
# Check the top 5 rows
users.head()

In [0]:
# Check the file info
print(users.shape)

Isso confirma que existem 6040 usuários e temos cinco recursos para cada um (ID de usuário exclusivo, sexo, idade, ocupação e o CEP em que eles moram).

### Movies Dataset

In [0]:
# Reading movies file
movies = pd.read_csv('https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/dataset/movies.csv', 
                     sep='\t', encoding='latin-1', 
                     usecols=['movie_id', 'title', 'genres'])

In [0]:
# Check the top 5 rows
movies.head()

In [0]:
# Check the file info
print(movies.shape)

Este Dataset contém atributos dos 3883 filmes. Existem 3 colunas, incluindo o ID do filme, seus títulos e gêneros. 

Os gêneros são separados por | e são selecionados entre 18 gêneros (Ação, Aventura, Animação, Infantil, Comédia, Crime, Documentário, Drama, Fantasia, Noir, Terror, Musical, Mistério, Romance, Ficção científica, Thriller, Guerra, Ocidental).

## Exploração dos Dados

### Ratings

Vamos dar uma olhada na distribuição dos ratings da nossa base

In [0]:
# Display distribution of rating
sns.countplot(ratings['rating'])

Parece que os usuários são bastante generosos em suas classificações. 
A classificação média é de 3,58 em uma escala de 5. Metade dos filmes tem uma classificação de 4 e 5. 

Vamos também dar uma olhada em um subconjunto de 10 filmes com a classificação mais alta e mais baixa.

In [0]:
# Unifica todos os datasets
dataset = pd.merge(pd.merge(movies, ratings),users)

# Tira a média dos ratings e filtra apenas filmes com no mínimo 50 avaliações
dataset = dataset.groupby(['title','genres'])\
  .agg({'rating': 'mean', 'user_id': 'count'})\
  .reset_index()

dataset = dataset[dataset.user_id > 50]

# Exibe os filmes com melhores notas
#
dataset[['title','genres','rating', 'user_id']]\
  .sort_values('rating', ascending=False)\
  .head(15)

In [0]:
# Exibe os filmes com pior nota 
#
dataset[['title','genres','rating']]\
  .sort_values('rating', ascending=True)\
  .head(15)

### Gêneros

A variável gêneros certamente será importante ao criar os mecanismos de recomendação, pois descreve o conteúdo do filme (ou seja, Animação, Horror, Ficção científica). Um pressuposto básico é que filmes do mesmo gênero devem ter conteúdos semelhantes. Vamos tentar ver exatamente quais gêneros são os mais populares.

In [0]:
movies.head(3)

In [0]:
# Gera a lista dos gereos de cada filme
df_genres = movies.genres.str.split("|", n = 5, expand = True)
df_genres = pd.concat([df_genres[n] for n in range(5)])
df_genres.head()

In [0]:
plt.figure(figsize=(10,5))
chart = sns.countplot(df_genres)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
chart

## Tipos de Recomendação

![rec-systems](https://github.com/khanhnamle1994/movielens/raw/cb1fe40c99cdd61c3c714e501e11f699c87b0eed/images/rec-systems.png)


## Modelo de Content-Based

Vamos construir uma Engine de Recomendação baseado em conteúdo que calcule a semelhança entre filmes com base em gêneros de filmes. Ele sugerirá filmes que são mais parecidos com um filme específico, com base em seu gênero. Para isso, utilizarei o arquivo ** movies.csv **.



In [0]:
movies.head()

In [0]:
movies.shape

#### Vamos criar o vetor de característica do ITEM

Para transformar a coluna em vetor de característica usarei a função **CountVectorizer** do **scikit-learn**, que transforma o texto em vetores de recursos que podem ser usados como entrada para o estimador.

In [0]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# 'a|b|c' -> ['a', 'b', 'c']
#
# 
tf = CountVectorizer()
tf_matrix = tf.fit_transform(movies['genres']).todense()
tf_matrix = pd.DataFrame(tf_matrix, columns = sorted(tf.vocabulary_.keys()))

tf_matrix.head()

In [0]:
tf_matrix.shape

Usarei o **[Cosine Similarity] (https://masongallo.github.io/machine/learning,/python/2016/07/29/cosine-similarity.html)** para calcular uma quantidade numérica que indica a semelhança entre dois filmes. 

In [0]:
cosine_sim = 1 - pairwise_distances(tf_matrix, metric='cosine')
cosine_sim[:4, :4]

In [0]:
cosine_sim.shape

Agora eu tenho uma matriz de similaridade de cosseno pareada para todos os filmes no conjunto de dados. O próximo passo é escrever uma função que retorne os 20 filmes mais semelhantes com base na pontuação de similaridade do cosseno.

![alt text](https://miro.medium.com/max/387/0*bESAibWjPtIQFRfR)

In [0]:
# Build a 1-dimensional array with movie titles
titles  = movies['title']
indices = pd.Series(movies.index, index=movies['title'])
  
# Busca filme pela substring
def search(substr):
  return movies[movies.title.str.contains(substr)]

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx        = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Vamos tentar obter as principais recomendações para alguns filmes e ver como as recomendações são boas.

In [0]:
search("Toy Story")

In [0]:
genre_recommendations('Toy Story (1995)').head(10)

In [0]:
genre_recommendations("Star Wars: Episode IV - A New Hope (1977)").head(10)

### Conclusão 

Como você pode ver, eu tenho uma lista bastante decente de recomendações para **Toy Story** e **Star Wars**.

No geral, aqui estão os pós do uso da recomendação baseada em conteúdo:

* Não há necessidade de dados sobre outros usuários, portanto, não há problemas de Cold Start e de escarsidade.
* Pode recomendar a usuários com gostos únicos.
* Pode recomendar itens novos e impopulares.
* Pode fornecer explicações para os itens recomendados, listando os recursos de conteúdo que fizeram com que um item fosse recomendado (nesse caso, gêneros de filmes)

No entanto, existem alguns contras de usar essa abordagem:
* Encontrar as características apropriadas é difícil.
* Não recomenda itens fora do perfil de conteúdo de um usuário.
* Incapaz de explorar julgamentos de qualidade de outros usuários.

## Modelo de Filtragem Colaborativa

Nesta seção, usaremos a Filtragem colaborativa baseada em memória para fazer recomendações aos usuários de filmes. A técnica baseia-se na ideia de que usuários semelhantes a mim podem ser usados para prever o quanto eu gostarei de um produto ou serviço específico que esses usuários usaram / experimentaram, mas eu não.


### Implementação

Usaremos o arquivo **ratings.csv** primeiro, pois ele contém ID do usuário, ID do filme e classificações. Esses três elementos são tudo o que preciso para determinar a semelhança dos usuários com base em suas classificações para um filme em particular.

Primeiro, faço um rápido processamento de dados:

In [0]:
ratings.head()

Devido ao poder de computação limitado, construiremos o sistema de recomendação usando apenas um subconjunto das classificações. Em particular, tirarei uma amostra aleatória de 20.000 classificações (2%) das classificações da 1M.

In [0]:
# Randomly sample 1% of the ratings dataset
#ratings = ratings#.sample(frac=0.02)
# Check the sample info
print(ratings.shape)

### Matriz de Interação USUARIO X ITEM

In [0]:
from scipy.sparse import csr_matrix

# pivot ratings into movie features
df_mf = ratings.pivot(
    index='user_id',
    columns='movie_id',
    values='rating'
).fillna(0)

df_mf.shape

# convert dataframe of movie features to scipy sparse matrix
#df_mf = csr_matrix(df_movie_features.values)
#df_mf

In [0]:
df_mf.head()

### Realizar alguma recomendação buscando os vizinhos

![alt text](https://miro.medium.com/max/319/0*e3_p3-HIh8DCin4m)

In [0]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric='cosine', 
                             algorithm='brute', 
                             n_neighbors=10, n_jobs=-1)

model_knn.fit(df_mf)

![alt text](https://image.slidesharecdn.com/annpresentation-150925020250-lva1-app6891/95/approximate-nearest-neighbor-methods-and-vector-models-nyc-ml-meetup-6-638.jpg?cb=1445710532)

#### Busca os vizinhos mais próximos do usuário 

In [0]:
# Busca os vizinhos mais próximos do usuário 
#
distances, indices = model_knn.kneighbors([df_mf.loc[10]], n_neighbors=10)

distances, indices

#### Lista os usuários mais próximos

In [0]:
# Lista os usuários mais próximos
user_neighbors = df_mf.iloc[indices[0]]
user_neighbors

#### Usa as pessoas similares para ponderar as notas dos filmes e seleciona K filmes para recomendar

In [0]:
# Usa as pessoas similares para ponderar as notas dos filmes e seleciona K filmes para recomendar
#
# Pode ser usada a média, mediana, máximo.. edt
#
#

rec_movies = user_neighbors.mean().sort_values(ascending=False)[:10]
rec_movies

#### Recomendação dos K melhores filmes para o usuário 

In [0]:
# Recomendação dos K melhores filmes para o usuário 
#
movies[movies.movie_id.isin(rec_movies.index)]

### Função de Recomendação para o Usuário

In [0]:
# Recomendação de filmes para um usuário específico com base no colaborativo
#
# user_id:   ID do usuário
# top:       Total de recomendações
# neighbors: Total de vizinhos analisados
# func: Aggregação dos vizinhos

def recommender_by_user_id(user_id, top=10, neighbors=10, func=np.mean):
  
  # Busca os vizinhos mais próximos do usuário 
  #
  distances, indices = model_knn.kneighbors([df_mf.loc[user_id]], n_neighbors=10)

  # Lista os usuários mais próximos
  user_neighbors = df_mf.iloc[indices[0]]

  # Usa as pessoas similares para ponderar as notas dos filmes e seleciona K filmes para recomendar
  rec_movies = user_neighbors.agg(func).sort_values(ascending=False)[:top]

  # Recomendação dos K melhores filmes para o usuário 
  #
  return movies[movies.movie_id.isin(rec_movies.index)]

In [0]:
# O Usuário 10 Tem um gosto variado entre Action, Sci-Fi e Drama
user_id = 10

recommender_by_user_id(user_id)

In [0]:
#pd.merge(ratings[ratings.user_id == user_id], movies)

In [0]:
# O Usuário 110 Tem um gosto bem específico para a categoria Crime, Drama, 
user_id = 110

recommender_by_user_id(user_id)

### Conclusão 

Como você pode observar, a recomendação por filtragem colaborativa gera recomendações bem variadas e é uma alteranativa muito melhor quando é possível ter acesso ao histórico de interações.

* O principal pós é justamente não precisar utilizar informações de conteúdo para fazer as recomendações, utilizando apenas as interações.

* O contra é a necessidade de ter muitos dados de histórico para que a filtragem seja de fato efetiva. 


## Engine RecSys

Agora, com os modelos de Content-Based e Colaborative Filtering tentem fazer uma função que dado o usuário ele retorne 5 recomendações baseadas em conteúdo e 5 recomendações de filtragem colaborativa:

In [0]:
# Recomendação completa usando a filtragem colaborativa e o content based.
# 
# Return DataFrame
def geral_recommender(user_id):
  pass



---



![slide1](https://storage.googleapis.com/ms_geral/workshop_recomendacao_campus/CampusParty%20-%20Sistemas%20de%20Recomenda%C3%A7%C3%A3o%20-%20Workshop-26.jpg?a=1)

## Outras alternativas 

Existem diversas alterantivas para recomendaçõo de conteúdo. Segue algumas:

* **Matriz de fatoração (MF)**: A idéia por trás de tais modelos é que as atitudes ou preferências de um usuário possam ser determinadas por um pequeno número de fatores latentes ocultos. Esses fatores também são chamados de **Embeddings**, que representam características diferentes para usuários e itens. A fatoração matricial pode ser feita por vários métodos, incluindo decomposição de Vecot de suporte (SVD), fatoração matricial probabilística (PMF) e fatoração matricial não negativa (NMF).



***Algoritmo baseado em cluster (KNN)***: A idéia de cluster é a mesma dos sistemas de recomendação baseados em memória. Em algoritmos baseados em memória, usamos as semelhanças entre usuários e / ou itens e as usamos como pesos para prever uma classificação para um usuário e um item. A diferença é que as semelhanças nessa abordagem são calculadas com base em um modelo de aprendizado não supervisionado, em vez da correlação de Pearson ou semelhança de cosseno.


***Redes Neurais/Aprendizado Profundo***: A idéia de usar Redes Neurais é semelhante à da Fatoração de Matrizes Baseada em Modelos. Na fatoração da matriz, decompomos nossa matriz esparsa original em produto de 2 matrizes ortogonais de baixo escalão. Para a implementação da rede neural, não precisamos que eles sejam ortogonais, queremos que nosso modelo aprenda os valores da própria matriz de incorporação. Os recursos latentes do usuário e os recursos latentes do filme são consultados nas matrizes de incorporação para uma combinação específica de filme e usuário. Estes são os valores de entrada para outras camadas lineares e não lineares. Podemos passar essa entrada para várias camadas relu, lineares ou sigmóides e aprender os pesos correspondentes por qualquer algoritmo de otimização (Adam, SGD, etc.).


![memory-model-cf](https://github.com/khanhnamle1994/movielens/raw/cb1fe40c99cdd61c3c714e501e11f699c87b0eed/images/memory-model-cf.jpg)

## Referências

* https://github.com/khanhnamle1994/movielens/
* https://medium.com/data-hackers/deep-learning-para-sistemas-de-recomendação-parte-1-introdução-b19a896c471e
